In [7]:
from scripts.models import VehTraj, PedTraj
from esper.widget import vgrid_widget
from vgrid import VideoBlockFormat, NestedFormat, NamedIntervalSet, IntervalBlock
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pyro
import pyro.distributions as dist
from pyro.poutine import trace
import math
from tqdm import tqdm_notebook as tqdm
from scripts.utils import img_grid, vgrid_traj, make_batch
import pandas as pd
from rekall import IntervalSetMapping, IntervalSet, Interval, Bounds3D
from enum import Enum
from torch import tensor
import torch
from pyro import plate
from pyro.infer import SVI, Trace_ELBO, TraceEnum_ELBO, JitTraceEnum_ELBO, config_enumerate
from pyro.optim import Adam
from pprint import pprint
from torch.distributions import constraints
from copy import copy
import re
from scripts import pattern
pyro.enable_validation(True)

In [2]:
vehicles = list(Vehicle.objects.all())
veh_trajectories = [VehTraj(veh) for veh in vehicles]

pedestrians = list(Pedestrian.objects.all())
ped_trajectories = [PedTraj(ped) for ped in pedestrians]

In [5]:
def model(N):
    ncat = 4
    transition_probs = pyro.sample(
        'transition_probs', 
        dist.Dirichlet(0.9 * torch.eye(ncat) + 0.1).to_event(1))
    
    state = 0
    for i in pyro.markov(N):
        state = pyro.sample('state_{}'.format(i), dist.Categorical(transition_probs[state]), 
                            infer={"enumerate": "parallel"})
        pyro.sample('vel_{}'.format(i))

In [223]:
vehicle = sample(uniform(Vehicle))
start, end = sample(uniform_interval(vehicle.start, vehicle.end))
stopping_car = sample(TrajectoryRegex('<+_'))
observe(stopping_car - vehicle.trajectory()[start:end] = 0)

obj = sample(uniform(Object))
time = vehicle.trajectory()[end].frame
observe(||obj.pos - time.pos|| = 0)

In [240]:
vehicle = pattern.Vehicle()
stopping_car = vehicle.trajectory().window(minlen=10, maxlen=30).filter(pattern.slowing_down)
close_obj = pattern.Object()
cars_stopping_for = \
    stopping_car.end().time() \
    .match(pattern.Close(close_obj, vehicle))

traces, _ = cars_stopping_for.eval(video=Video.objects.all()[3])
vgrid_traj([trace[stopping_car] for trace in traces[:1]])

VGridWidget(vgrid_spec={'database': {'videos': [{'path': '/app/data/vci-dataset-dut/data/videos/intersection_0…

In [231]:
vehicle = pattern.Vehicle()
turning_car = vehicle.trajectory().match(pattern.TurningLeft())

traces, scores = turning_car.eval()
vgrid_traj([trace[turning_car] for trace in traces[:10]])

VGridWidget(vgrid_spec={'database': {'videos': [{'path': '/app/data/vci-dataset-dut/data/videos/intersection_1…

In [233]:
vehicle = pattern.Vehicle()
pedestrian = pattern.Pedestrian()
fast_car = vehicle.trajectory().match(pattern.Speeding())


AttributeError: module 'scripts.pattern' has no attribute 'Speeding'